In [ ]:
# Download the dataset if you haven't already

!wget -O data/dataset.rar https://github.com/madlabub/Machining-feature-dataset/blob/master/dataset.rar?raw=true
!unrar x data/dataset.rar data/
!rm -r data/dataset.rar 

In [ ]:
#importing the necessary libraries

import os
import torch
import numpy as np
import pickle
import open3d as o3d
from model.resunet import ResUNetBN2C
from util.visualization import get_colored_point_cloud_feature
from numpy import linalg as LA
from util.feature_extraction import stl2ply_convert, feature_extract
#from util.feature_extraction import get_features
from util.misc import sort_list, zero_pad
from tqdm import tqdm

In [6]:
# loading the pretrained fcgf model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load('model/pretrained/model_32.pth', map_location=device)
model = ResUNetBN2C(1,32, normalize_feature=True, conv1_kernel_size=checkpoint["config"]["conv1_kernel_size"], D=3)
model.load_state_dict(checkpoint['state_dict'])
model.eval()
model = model.to(device)

### Loading Data

In [7]:
# Here the data is available is STL format

stl_folder = "data/stl"

ply_folder = "data/ply"

if not os.path.exists(ply_folder):
    os.makedirs(ply_folder)


### Convertion to PLY

In [8]:
# Converting .stl files to .ply format for feature extraction

stl2ply_convert(stl_folder,ply_folder)

### Feature Extraction

In [9]:
#Feature Extraction from .ply files

file_names, file_paths, features = feature_extract(ply_folder, model, device)


/home/ubuntu/kamal/shape_search_new/util/feature_extraction.py:179: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  coords = torch.tensor(coords, dtype=torch.int32)


In [10]:
#Printing the different classes
unique_list = []

# traverse for all elements
for x in file_names:
    # check if exists in unique_list or not
    if x not in unique_list:
        unique_list.append(x)

for x in unique_list:
    print(x)
     

9_triangular_pocket
20_v_circular_end_blind_slot
11_circular_end_pocket
4_rectangular_passage
21_h_circular_end_blind_slot
0_Oring
16_2sides_through_step
2_blind_hole
23_6sides_pocket
7_rectangular_through_slot
6_triangular_through_slot
17_slanted_through_step
22_6sides_passage
8_rectangular_blind_slot
1_through_hole
5_circular_through_slot
3_triangular_passage
15_rectangular_through_step
19_round
14_rectangular_blind_step
18_chamfer
13_circular_blind_step
12_triangular_blind_step
10_rectangular_pocket


In [11]:
# It is adviced to pickle the extracted features to save time in execution next time

data = [file_names, file_paths, features]
with open("data/names_paths_features_featurenet.dat", "wb") as f:
    pickle.dump(data, f) 
    
#Unpickling the features

with open("data/names_paths_features_featurenet.dat", "rb") as f:
    file_names, file_paths, features = pickle.load(f)


In [12]:
print("shape of a sample feature vector:",features[0].shape)

shape of a sample feature vector: (14, 32)


### Benchmarking

In [13]:
N_files = len(features)

tested_files = list()
sim_list = list()
sorted_list = list()
check_1 = 0
check_5 = 0
for i in range(0,N_files):
    test_feature = features[i]
    tested_files.clear()
    sim_list.clear()
    for j in range(0,N_files):
        if file_names[j] not in tested_files:
            tested_files.append(file_names[j])
            db_feature = features[j]
            # finding out the similarity between features
            sim_list.append(abs(LA.norm(test_feature, 'fro') - LA.norm(db_feature, 'fro')))
    sorted_list = sort_list(file_names, sim_list)
    check_1 += (file_names[i] ==  sorted_list[0])
    if file_names[i] in sorted_list[0:4]:
        check_5 += 1     

In [14]:
print("Top-5 Accuracy with Frobenius Norm is "+"\t",check_5/N_files)
print("Accuracy with Frobenius Norm is is "+"\t",check_1/N_files)

Top-5 Accuracy with Frobenius Norm is 	 0.847875
Accuracy with Frobenius Norm is is 	 0.311


We can observe the inability of Frobenius norm by itslef to serve as a good similarity measure.
